In [1]:

#Objectifs:
#  Programmer sous R:
#    1) Une fonction pour la (TRAIN-TEST) validation répétée
#    2) Une fonction pour la validation croisée (K feuilles)
#    3) Une fonction pour la validation croisée imbriquée
#    4) Une fonction pour le Leave One Out (avec k = n)
# 
# deux critères à valider: moyenne des performances et écart-type des 

# construction de modèles (ou librairies de notre choix):
# library(rpart) = algos de décisions -> CART: classification and regression tree (classif ou regression)
# library(MASS) = algo de décisions -> LDA : linear decision analysis: (classif binaire)
# library(FNN) : K plus proche voisin (k-PPV)
# tree <- rpart(X10 ~ ., data= mesData, method= 'class')  ici ~ = en fonction de et . = toutes les autres variables
# tree <- rpart(X10 ~ ., data= mesData, method= 'class', control=rpart.control(minsplit=50), cp= val entre 0 et 1)  ici ~ = en fonction de et . = toutes les autres variables
# predict(tree, mesNouvellesDonnées, type='class')
# predict(tree, mesNouvellesDonnées, type='prob')
# JEUX de données madoc: Breiman: wave5000

# attente livrable: les fonctions, mini rapport explication fonction et comparaison des modèles(1,2,3,4) :robustesse sur les données de Breiman

# en sortie un vecteur mélanger de 1 à n: en entré un entier
# fonction sample déjà défini


# Question 1
#data = dataframe avec une colonne réel
#k = le nombre de répétition
#predictColumn = string = nom de la colonne X10
library(rpart)
validationRepeteeRPART = function (data, k) {
  errorRate = c();
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData$X10~.,modalData, method="class")
    prediction = predict(tree, testData, type="class")
    
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction)
    
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

library(MASS)

validationRepeteeLDA = function (data, k) {
  errorRate = c()
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = lda(modalData$X10~.,modalData, method="moment")
    prediction = predict(tree, testData, type="moment")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction$class)
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

#Exercice 2
#data = dataframe
#partitions = integer
validationCroiseeRPART = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData_step2$X10~.,modalData_step2, method="class")
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}




validationCroiseeLDA = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = lda(X10~.,modalData_step2)
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction$class)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}


#appel de fonction validationCroisee défini précédemment
#en paramètre k = le nombre de répétition
validationCroiseeRepetee = function (data, partitions, k) {
  robustesseMean = 0
  errorRateMean = 0
  
  for (i in 1:k) {
    robustesseMean = robustesseMean + validationCroiseeRPART(data, partitions, 1)
    errorRateMean = errorRateMean + validationCroiseeRPART(data, partitions, 0)
  }
  
  robustesseMean = robustesseMean/k
  errorRateMean = errorRateMean/k
  
  return(c(robustesseMean, errorRateMean))
  
}

#data = dataframe
#partitions = nombre de partition où on va split le dataset
validationCroiseeImpliquee = function(data, partitions) {
  
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  listErrorRate = c()
  
  for (i in 1:partitions) {
    
    errorRate = c()
    
    for (i in 1:partitions-1) {
 
      tempSetPartitions = setPartitions
      currentTestData = setPartitions[[i]]
      currentModalDataList = tempSetPartitions[-i]
      #fusion dataframe
      currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
      
      for (j in 3:(partitions-2)) {
        currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
      }
      
      numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
      numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
      
      testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
      modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
      
      #on calcule la précision (le taux d'erreurs)
      tree = lda(X10~.,modalData_step2)
      prediction = predict(tree, testData_step2, type="class")
      #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
      contingencyTable = table(testData_step2$X10, prediction$class)
      
      total = 0
      sumError = 0
      for (i in 1:nrow(contingencyTable)) {
        for (j in 1:ncol(contingencyTable)) {
          if (i != j) {
            sumError = sumError + contingencyTable[i,j]
          }
          total = total + contingencyTable[i,j]
        }
      }
      errorRate = c(errorRate, c(sumError/total))
    }
    
    listErrorRate = c(c(listErrorRate), mean(errorRate))
    
  }
  
  return(listErrorRate)
  
}

#appel de la fonction validationCroisee
#en paramètre data: un dataframe
leaveOneOut = function(data) {
  numberOfrows = nrow(data)
  return(c(validationCroiseeRPART(data, numberOfrows, 0), validationCroiseeRPART(data, numberOfrows, 1)))
}


#appel de tous les fonctions définies précédemment

algorithmSelection = function(data, K) {
  
  validationCroiseeRPARTErrorRate = validationRepeteeLDA(data, K)
  validationCroiseeLDAErrorRate = validationRepeteeRPART(data, K)
  
  if (validationCroiseeRPARTErrorRate > validationCroiseeLDAErrorRate) {
    return("SOLUTION: LDA")
  } else {
    return("SOLUTION: RPART")
  }
  
  return("ERROR")
}

validationCroiseeGetModel = function(data, partitions, colonneCible, errorRateThreshold = 0.3) {
  errorRate = c()
  #on m\u{fffd}lange les donn\u{fffd}es
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la pr\u{fffd}cision (le taux d'erreurs)
    #formulaString = paste("modalData_step2$", colonneCible, "~.",sep="")
    #formulaCible= as.formula(formulaString)
    #tree = rpart(formulaCible, modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
      
      
    #Ici on s'assure que notre vecteur "cible" ne contient pas uniquement des 0.
    #Cela peut arriver pour la prediction des dernieres variables (secteur 1, 2 et particulier), qui sont composés de beaucoup de 0
    #Si jamais le modalData_step["variableCible"] contient que des valeurs identiques (ici 0), R nous signale qu'il n'y a aucun interet à créer un modèle et retourne une erreur
    while(sum(modalData_step2[1]) == 0 && sum(testData_step2[1]) == 0){
        
        #Regenerer les modalData et testData 
        randomData_step2 = data[sample(nrow(data)),]
        testData_step2 = randomData_step2[1:numOfLinesTestCurrentModalData,]
        modalData_step2 = randomData_step2[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    }

    tree = rpart(modalData_step2$X1~., modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
    prediction = predict(tree, testData_step2, type="class")
      
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    #formulaString = paste("testData_step2", colonneCible,sep="$")
    #formulaCible2 = as.formula(formulaString)
      
    contingencyTable = table(testData_step2$X1, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la m\u{fffd}me que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (mean(errorRate) <= errorRateThreshold) {
    listResult = list(mean(errorRate), tree)
    print("Error Rate < 0.3")
    print(mean(errorRate))
    return(listResult)
  }else{
    listResult = list(0, tree)
    print("Error Rate > 0.3, WRONG MODEL")
    print(mean(errorRate))
    return(listResult)
  }
}

#Variable cible : PREDICTION CHIFFRE D'AFFAIRES
GetModelChiffreAffaires = function(data, partitions){
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,9)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionChiffreAffaire = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionChiffreAffaire)
  return (predictionChiffreAffaire)
}

#Variable cible : CAPACITE D'EMPRUNT
GetModelCapaciteEmprunt = function(data, partitions){
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.6)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionCapaciteEmprunt = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionCapaciteEmprunt)
  return (predictionCapaciteEmprunt)
}

GetModelSecteurOne = function(data, partitions){
#Variable cible : SECTEUR 1
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,3,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurOne = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurOne)
  return (predictionSecteurOne)
}

GetModelSecteurTwo = function(data, partitions){
#Variable cible : SECTEUR 2
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurTwo = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurTwo)
  return (predictionSecteurTwo)
}

GetModelSecteurParticulier = function(data, partitions){
#Variable cible : SECTEUR PARTICULIER
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurParticulier = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurParticulier)
  return (predictionSecteurParticulier)
}

In [2]:
apprentissage <- read.table("apprentissage.csv", sep = "," ,header = FALSE)
test <- read.csv("test.csv", sep = "," ,header = FALSE)
# Remplacer les NA par des 0
apprentissage[is.na(apprentissage)]<-0
test[is.na(test)]<-0

#Option permettant de désactiver les warnings pour une meilleure visibilité des résultats
oldw <- getOption("warn")
options(warn = -1)
predictionChiffreAffaire = GetModelChiffreAffaires(apprentissage, 10)
options(warn = oldw)


[1] "Error Rate < 0.3"
[1] 0.06666667
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 19 0 (0.24 0.08 0.08 0.04 0.04 0.08 0.04 0.04 0.04 0.08 0.04 0.08 0.04 0.08)  
   2) X2< 0.5 6  0 0 (1 0 0 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 19 17 17500 (0 0.11 0.11 0.053 0.053 0.11 0.053 0.053 0.053 0.11 0.053 0.11 0.053 0.11)  
     6) X2< 10.5 17 15 17500 (0 0.12 0.12 0.059 0.059 0.12 0.059 0.059 0.059 0.12 0.059 0.12 0.059 0)  
      12) X2< 3.5 11  9 17500 (0 0.18 0.18 0.091 0.091 0.18 0.091 0.091 0.091 0 0 0 0 0)  
        24) X8>=88.615 5  3 17500 (0 0.4 0.4 0.2 0 0 0 0 0 0 0 0 0 0)  
          48) X9< 38.865 3  1 20000 (0 0.33 0.67 0 0 0 0 0 0 0 0 0 0 0) *
          49) X9>=38.865 2  1 17500 (0 0.5 0 0.5 0 0 0 0 0 0 0 0 0 0) *
        25) X8< 88.615 6  4 35000 (0 0 0 0 0.17 0.33 0.17 0.17 0.17 0 0 0 0 0)  
          50) X11< 219.46 3  1 35000 (0 0 0 0 0 0.67 0 0.33 0 0 0 0 0 0) *
          51) X11>=219.46 3  2 28500 (0 0 0 0 0.33 0 0.33 0 0.33 0 0 

In [3]:
oldw <- getOption("warn")
options(warn = -1)
predictionCapaciteEmprunt = GetModelCapaciteEmprunt(apprentissage, 10)
options(warn = oldw)

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.7
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.85
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.7
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8166667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "Er

In [4]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurOne = GetModelSecteurOne(apprentissage, 10)
options(warn = oldw)

[1] "Error Rate < 0.3"
[1] 0.3
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 25 3 0 (0.88000000 0.12000000)  
  2) X14< 2414.395 23 1 0 (0.95652174 0.04347826) *
  3) X14>=2414.395 2 0 1 (0.00000000 1.00000000) *


In [5]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurTwo = GetModelSecteurTwo(apprentissage, 10)
options(warn = oldw)


[1] "Error Rate < 0.3"
[1] 0.03333333
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 25 2 0 (0.9200000 0.0800000)  
  2) X5< 1947.47 23 0 0 (1.0000000 0.0000000) *
  3) X5>=1947.47 2 0 1 (0.0000000 1.0000000) *


In [6]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurParticulier = GetModelSecteurParticulier(apprentissage, 10)
options(warn = oldw)

[1] "testData"
      X1
3011   0
6620   0
10689  0
3808   0
10175  0
10421  0
[1] "modalData"
      X1
6772   0
11229  0
4821   0
746    0
290    0
1402   0
9811   0
9352   0
2379   0
6541   0
7391   0
6489   0
6928   0
3802   0
9201   0
8451   0
52     0
2650   0
227    0
6778   0
9698   0
3713   0
1713   0
11495  0
2943   0
[1] "testData"
      X1
10199  0
4297   0
7443   0
4225   0
6111   0
5186   0
[1] "modalData"
      X1
10576  0
3880   0
2971   0
3173   0
3890   0
8295   0
5142   0
3736   0
11383  0
4718   0
1138   0
3562   0
4236   0
1784   0
1062   0
6933   0
9013   0
8258   0
11301  0
4957   0
10381  0
10359  0
10117  0
6380   0
8513   0
[1] "testData"
      X1
9069   0
2073   0
9417   0
7172   0
10788  0
8210   0
[1] "modalData"
      X1
10332  0
349    0
2320   0
8512   0
7752   0
4724   0
7616   0
6621   0
964    0
9064   0
7539   0
9177   0
4455   0
6033   0
6059   0
11199  0
5699   0
1259   0
7534   0
2060   0
8522   0
9034   0
11496  0
5639   0
1017   0
[1] "testData"
 

In [7]:
resultPrediction = data.frame(test[ , -c(1,2,3,35,36,37,38,39)], predictionChiffreAffaire, predictionCapaciteEmprunt, predictionSecteurOne, predictionSecteurTwo, predictionSecteurParticulier)
write.csv(resultPrediction, file = "resultPrediction.csv",row.names=TRUE)